In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import os
import pickle as pkl
from collections import Counter, defaultdict
from itertools import combinations

import numpy as np
import scipy as sp
from tqdm import tqdm
from scipy import sparse
from scipy.sparse import linalg
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

import numpy as np
from sklearn.manifold import TSNE
import glob

from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
frame = pd.read_csv("/data/shruti/conspirecovery/reddit/initial/network/subLinkScores.gz", 
                    compression='gzip', header=0)

frame.head()

,url,f0_,f1_
0,https://www.youtube.com/watch?v=7F2hVzdRdJk&am...,"NSFL_,nsfl_videos,MorbidReality,Health,nsfw_wtf","3,5,0,1,1"
1,https://www.independent.co.uk/environment/chic...,"UKNewsByABot,u_thegrandspatulass,u_theindepend...","1,1,1,2,1,6,362.5,68"
2,http://thelesboporn.com/hardcore/head-first-ev...,"NSFW_Lesbian_Porn,NSFW_LESBIAN_LUST,LesbianGsp...","2,1,1,1,1"
3,http://i.imgur.com/6ElZKZS.jpg,"RealGirls,sexycougar,GoneWildHairy,gonewild,Am...","1.5,26,616,330,5.5,2697,1,1468,50,3"
4,http://mouthbreathercult.com/,"flocked,Mathcore,Metalcore,experimentalmusic,l...","1,12,2,1,1,1"


In [3]:
frame.sample(5)

,url,f0_,f1_
325534,https://i.redd.it/wrnwdwzgoy421.jpg,"MadeMeSmile,wholesomememes,LibertarianUncensor...","54927,1206,82,19,2,69,290"
1537022,https://mobile.twitter.com/realDonaldTrump/sta...,"ResponseToPresident,TrumpTheGreat,TheTwitterFe...","1,1,1,3,1"
1768939,https://www.theguardian.com/commentisfree/2017...,"WayOfTheBern,UMukhasimAutoNews,RedditSample,Ec...","12,1,1,1,1"
1045275,https://www.youtube.com/watch?v=eMQStMo0iEg,"PromoteGamingVideos,CoD_wwii,Call_of_Duty_WWII...","0,3,4,1,1"
927594,https://gfycat.com/perfectwiltedcaterpillar,"interestingasfuck,perfectloops,mildlyinfuriati...","215,8301,4,1,1,36,36,36,76,135.5,103"


In [4]:
meanSTDs = pd.read_csv("/home/phadke/recovery/conspirecovery/lite_data/subScoreAvgStd.csv", header=0)
meanSTDs.head()

,subreddit,meanScore,stdScore
0,slabserver,5.900498,4.976567
1,MLBB,2.028571,1.230231
2,JJAbrams,1.300000,0.640312
3,DarcMatterOfficial,8.061474,16.306947
4,The22Scoops,1.500000,0.802955


In [5]:
sub2mean = meanSTDs.set_index('subreddit').to_dict()['meanScore']
sub2std  = meanSTDs.set_index('subreddit').to_dict()['stdScore']

In [6]:
def getZscored(row):
    subs = row['f0_'].split(",")
    scrs = list(map(float, row['f1_'].split(",")))
    zscrs = []
    try:
        for i in range(len(scrs)):
            z_scr = (scrs[i] - sub2mean[subs[i]])/sub2std[subs[i]]
            zscrs.append(z_scr)

        return zscrs
    except:
        fakevar = 0

In [7]:
frame['zscores'] = frame.parallel_apply(lambda row: getZscored(row), axis=1)

In [8]:
frame.sample(5)

,url,f0_,f1_,zscores
97695,https://www.youtube.com/watch?v=ThEYMxlRFNA,"SmallYoutubers,gamingvids,SmallYTChannel,Adver...","1,1,1,1,1,1","[-0.15927383157131036, -0.07592580530305668, -..."
900663,https://i.imgur.com/uHxE2DG.png,"nsfw_snapchat_share,nsfw2,camwhores,snapchatgw...","28,200,134,62,1,1,1,1,98,100,61,3,1,88,1,1,39,...","[2.831809389256382, 1.6638071121304823, 1.5279..."
1673087,http://sextube4mobile.com/teen/lusty-blonde-ba...,"teen_porn,Teen_girlxxx,adulttube,HotPoonTube,s...","1,1,1,1,1","[-0.1851432461792231, -0.05786533205423014, -0..."
809867,http://m.imgur.com/a/g0uSh,"kinksters_gone_wild,WouldYouFuckMyWife,Swinger...","34,14,55,16,4,61,21","[-0.30435995082593686, -0.22397671153647475, -..."
1805383,https://gfycat.com/repentantweeklycow,"cock,penis,Perfectdick,selfservice,MalesMastur...","5,5,0,3,31,4","[-0.2088103688214328, -0.2386535272786093, -0...."


In [9]:
len(frame)

2593714

In [10]:
len(frame.dropna(subset=['zscores']))

2509394

In [11]:
frame = frame.dropna(subset=['zscores'])

In [12]:
groupScores = defaultdict(list)

for idx, row in frame.iterrows():
    rowsubs = row['f0_'].split(",")
    rowscores = row['zscores']
    if len(rowsubs)==len(rowscores):
    
        for i in range(len(rowsubs)-1):
            for j in range(i+1, len(rowsubs)):
                subtuple = sorted([rowsubs[i], rowsubs[j]])
                stuple = (subtuple[0], subtuple[1])
                subidx1 = rowsubs.index(subtuple[0])
                subidx2 = rowsubs.index(subtuple[1])
                if stuple not in groupScores:
                    groupScores[stuple] = [[],[]]

                groupScores[stuple][0].append(rowscores[subidx1])
                groupScores[stuple][1].append(rowscores[subidx2])
            
    

In [13]:
groupScores

KeyboardInterrupt: 

In [17]:
len(groupScores[('climateskeptics', 'conspiracy')][1])

374

In [23]:
# import pickle as pkl
# with open("/data/shruti/conspirecovery/reddit/initial/network/network_paied_scores.pkl", "wb") as pfile:
#     pkl.dump(groupScores, pfile)

In [24]:
frame.to_csv("/data/shruti/conspirecovery/reddit/initial/network/sublinkscore_zscored.csv", header=0)

In [2]:
with open("/data/shruti/conspirecovery/reddit/initial/network/network_paied_scores.pkl", "rb") as pfile:
    groupScores = pkl.load(pfile)

In [3]:
from scipy.stats import pearsonr

In [15]:
sublinkcorr = defaultdict()

for k in groupScores.keys():
    if len(groupScores[k][0]) > 15:
        corr = pearsonr(groupScores[k][0], groupScores[k][1])
        sublinkcorr[k] = corr

/home/phadke/.local/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [16]:
sublinkcorr[('climateskeptics', 'conspiracy')]

(0.43848070144740986, 5.268174814421419e-19)

In [17]:
tuplelists = list(zip(*list(sublinkcorr.keys())))

sublist = tuplelists[0] + tuplelists[1]

# for t in tuplelists:
#     sublist.append(t[0])
#     sublist.append(t[1])
    
sublist = list(set(sublist))

In [18]:
len(sublist)

33567

In [19]:
coocMat = np.zeros((len(sublist), len(sublist)),)

In [20]:
coocMat.shape

(33567, 33567)

In [21]:
for tplist in list(sublinkcorr.keys()):
    n1_idx = sublist.index(tplist[0])
    n2_idx = sublist.index(tplist[1])
    
    coocMat[n1_idx, n2_idx] = sublinkcorr[tplist][0]
    coocMat[n2_idx, n1_idx] = sublinkcorr[tplist][0]


In [22]:
coocMat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
coocMat.shape

(33567, 33567)

In [27]:
np.sum(coocMat[:,1])

0.12247634788585407

In [44]:
# with open("/data/shruti/conspirecovery/reddit/initial/network/coocMat.pkl", "wb") as pfile:
#     pkl.dump(coocMat, pfile)

In [12]:
# with open("/data/shruti/conspirecovery/reddit/initial/network/coocMat.pkl", "rb") as pfile:
#     coocMat = pkl.load(pfile)

In [13]:
coocMat

array([[ 0.        , -0.03338903,  0.        , ...,  0.03081668,
         0.        ,  0.        ],
       [-0.03338903,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.03081668,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [14]:
coocMat.shape

(63819, 63819)

In [28]:
def get_svd_matrix(pmi_matrix, embedding_size=200,
                   svd_diag_exponent=0.5):
    uu, ss, vv = linalg.svds(pmi_matrix, embedding_size)

    print('ppmi size: {}'.format(pmi_matrix.shape))
    print('embedding size: {}'.format(embedding_size))
    print('uu.shape: {}'.format(uu.shape))
    print('ss.shape: {}'.format(ss.shape))
    print('vv.shape: {}'.format(vv.shape))

    svd_word_vecs = uu.dot(np.diag(ss ** svd_diag_exponent))
    print(svd_word_vecs.shape)
    return svd_word_vecs

In [29]:
svdMat = get_svd_matrix(coocMat, embedding_size=200)

ppmi size: (33567, 33567)
embedding size: 200
uu.shape: (33567, 200)
ss.shape: (200,)
vv.shape: (200, 33567)
(33567, 200)


In [30]:
svdMat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [33]:
np.sum(svdMat[:,3])

0.0